# Generate qsub scripts for STEP03 operations

This notebook generates a set of PBS scripts and a bash script that submits all the PBS jobs. It works on the data analysis server of NSF NCAR: `casper.ucar.edu`.

In [1]:
import os
import yaml
import numpy as np

## Open The Config - ZES (6 hourly)

In [ ]:
config_name = os.path.realpath('verif_config.yml')

with open(config_name, 'r') as stream:
    conf = yaml.safe_load(stream)

## Set Number of Forecasts 

In [3]:
IND_max = 2*(365+366+365)
INDs = np.arange(0, IND_max+100, 100)
INDs[-1] = IND_max

## Generate Qsub Scripts for Forecast Model

In [5]:
for i, ind_start in enumerate(INDs[:-1]):
    
    ind_end = INDs[i+1]
    
    f = open('{}verif_ZES_WX_{:03d}.sh'.format(conf['qsub']['qsub_loc'], i), 'w') 
    
    heads = '''#!/bin/bash -l

#PBS -N ZES_MF
#PBS -A {project_code}
#PBS -l walltime=23:59:59
#PBS -l select=1:ncpus=4:mem=16GB
#PBS -q casper
#PBS -o verif_ZES_MF.log
#PBS -e verif_ZES_MF.err

conda activate {conda_env}
cd {}
python STEP03_ZES_ForecastModel.py {} {}
'''.format(project_code=conf['qsub']['project_code'],
           conda_env=conf['qsub']['conda_env'],
           scripts_loc=conf['qsub']['scripts_loc'], 
           ind_start=ind_start, 
           ind_end=ind_end)
    
    print(heads, file=f)    
    f.close()

f = open('{}step03_ZES_MF_all.sh'.format(conf['qsub']['qsub_loc']), 'w')

for i, ind_start in enumerate(INDs[:-1]):
    print('qsub verif_ZES_MF_{:03d}.sh'.format(i), file=f)
    
f.close()